# Stan w programowaniu obiektowym

## Zmienne instancyjne i klasowe, zasady dostępu do danych

<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@agh.edu.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe

## konsultacje: wtorek 15:30 - 18:00, pokój 4.61

<img src="img/java_log.png" style="height: 600px" />

# Sonda Mars Climate Orbiter

<img src="img/mars.jpg" width="700"/>

Przyczyną katastrofy sondo były użycie amerykańskich jednostek miar (funtów) w modelu odpowiedzialnym za obliczanie trajektorii lotu sondy. Pozostałe moduły używały jednostek SI i ciąg obliczany był w niutonach. Ta różnica doprowadziła do niewłaściwej korekty lotu i w efekcie katastrofy. Koszt misji wyniósł 326 milionów dolarów.

# Program strukturalny

In [ ]:
struct SpaceProbe {
    double spaceProbeSpeed;
    double spaceProbeWeight;
}

double thrustCorrectionForX(struct SpaceProbe spaceProbe){
    // operowanie na polach spaceProbeSpeed oraz spaceProbeWeight
}

double thrustCorrectionForY(struct SpaceProbe spaceProbe){
    // ...
}

double thrustCorrectionForZ(struct SpaceProbe spaceProbe){
    // ...
}

# Primitive obsession

In [ ]:
class SpaceProbe {
    private double speedX;
    private double speedY;
    private double speedZ;
    private double weight;
    
    public double thrustCorrectionForX(double positionX, 
                      double positionY, double positionZ){
        //...
    }
}

Używanie typów prymitywnych tam, gdzie mamy do czynienia z wartościami domenowymi. Tutaj składowe wektora prędkości są osobnymi polami i nie mają jednostki. Masa też nie ma jednostki.

# Klasa `Speed`

In [ ]:
class Speed {
    public int value;
    public String unit;
    
    public Speed(int value, String unit){
        this.value = value;
        this.unit = unit;
    }
}

* Klasa `Speed` definiuje dwa **pola**: value oraz unit. Służą one do przechowywania stanu obiektów klasy Speed. 
* Oba pola są *publiczne* oznacza to, że można je odczytywać i zapisywać *na zewnątrz* klasy. Oznacza to dokładnie, że każda inna klasa ma dostęp do tych pól.
* Klasa posiada również publiczny konstruktor, który akceptuje dwa parametry. Te parametry wykorzystywane są do
  *inicjalizacji* obiektów klasy `Speed`.
* W konstruktorze występuje słowo kluczowe `this` - pozwala ono odróżnić zmienne lokalne `value` i `unit` od 
  zmiennych *instancyjnych* `value` i `unit`. Poprzedzenie nazwy zmiennej tym słowem, oznacza, że odnosimy się 
  do zmiennych, których właścicielem jest obiekt.
* Jedna składowa wektora typu int została wprowadzona dla uproszczenia przykładu.

In [ ]:
Speed speed1 = new Speed(10, "km/h");
Speed speed2 = new Speed(20, "m/s");

* Dalej widzimy wykorzystanie konstruktora - słowo `new` służy do tworzenia nowych obiektów. 
  Wywołuje ono konstruktor z takimi samymi parametrami jak te zdefiniowane w klasie.
* Wywołanie `speed1.value = 20` zmienia **stan** obiektu `speed1`.

<img src="img/instance_variables.png" />

In [ ]:
speed1.value = 20;
speed2.unit = "km/h";

<img src="img/instance_variables_2.png"/>

In [ ]:
speed1.value = -10;
speed2.unit = "ala ma kota";

<img src="img/public_1.png"/>

## 1 ulepszenie - użycie typu wyliczeniowego

In [2]:
enum SpeedUnit {
    MS, KMH;
}

In [ ]:
class Speed {
    public int value;
    public SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}

In [ ]:
Speed speed1 = new Speed(10, SpeedUnit.KMH);
Speed speed2 = new Speed(20, SpeedUnit.MS);
System.out.println(speed1);

<img src="img/instance_variables_5.png"/>

# 2 ulepszenie - użycie modyfikatora `private`

In [ ]:
class SpaceProbe {
    private Speed speed;
    
    public SpaceProbe(Speed speed){
        this.speed = speed;
    }
    
    public Speed getSpeed(){
        return this.speed;
    }
}

In [ ]:
SpaceProbe probe1 = new SpaceProbe(new Speed(10, SpeedUnit.KMH));

probe1.speed = new Speed(10, SpeedUnit.MS);

In [ ]:
SpaceProbe probe1 = new SpaceProbe(new Speed(10, SpeedUnit.KMH));
Speed speed1 = probe1.getSpeed();

speed1.unit = SpeedUnit.MS;

System.out.println(probe1.getSpeed().unit);

W Javie wszystkie wartoście z wyjątkiem prymitywnych są zawsze przekazywane przez referencję.

# 3 ulepszenie -  modyfikator `final` - `Speed` jako ValueObject

In [ ]:
class Speed {
    public final int value;
    public final SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}

In [ ]:
SpaceProbe probe1 = new SpaceProbe(new Speed(10, SpeedUnit.KMH));
Speed speed1 = probe1.getSpeed();

speed1.unit = SpeedUnit.MS;

 <img src="img/room_temperature.png" width="300"/>

* ValueObject nie może zmienić swojej wartości - jest jak liczba, znak, etc. Przykładem może być konkretna temperatura 20 stopni C.
* Inny obiekt korzystający z ValueObject może zminić wartość *parametru wyrażanego za pomocą* ValueObject. Np. temperatura pokoju moze zmienić się z 20 na 22. Ale same valueObjects się nie zmienią. 

# Jak działa modyfikator `private`?

In [ ]:
class Speed {
    private int value;
    private SpeedUnit unit;

    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}

Tutaj usunęliśmy modyfikator `final` żeby pokazać jak działa modyfikator `private`.

In [ ]:
class SpaceProbe {
    private Speed speed;
    
    public void accelerate(Speed delta){
        speed.value += delta.value;
    }
}

* Obiekty "zewnętrzne" nie mogą **odczytywać** ani **zapisywać** wartości pól prywatnych. Ale co to znaczy?

In [ ]:
class Speed {
    private int value;
    private SpeedUnit unit;

    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
    
    public Speed add(Speed that){
        if(this.unit == that.unit){
            return new Speed(this.value + that.value, this.unit);
        } else {
            return null;
        }
    }
    
    public boolean isLower(int value){
        return this.value < value;
    }
}

In [ ]:
class SpaceProbe {
    private Speed speed;
    private static final int MAXIMUM_SPEED = 100;

    public void accelerate(Speed delta){
        Speed newSpeed = this.speed.add(delta);
        if(newSpeed.isLower(MAXIMUM_SPEED)){
            this.speed = newSpeed;
        }
    }
}

<img src="img/private.png"/>

# Modyfikator `protected`

In [3]:
class Speed {
    protected SpeedUnit unit;
    
    public Speed(SpeedUnit unit){
        this.unit = unit;
    }
    
    public Speed add(Speed delta){
        return this; // does nothing
    }
}

In [4]:
class Speed1D extends Speed {
    protected int value;

    public Speed1D(int value, SpeedUnit unit){
        super(unit);
        this.value = value;
    }
    
    public Speed add(Speed delta){
        if(delta instanceof Speed1D){
            Speed1D delta1d = (Speed1D) delta;
            if(this.unit == delta1d.unit){
                return new Speed1D(this.value + delta1d.value, this.unit);
            } else {
                return this;
            }
        } else {
            return this;
        }
    }
    
    public String toString(){
        return "(" + this.value + ") " + this.unit;
    }
}

In [5]:
Speed speed1 = new Speed1D(10, SpeedUnit.KMH);
Speed speed2 = new Speed1D(20, SpeedUnit.KMH);

Speed speed3 = speed1.add(speed2);
System.out.println(speed3);

(30) KMH


In [6]:
class Speed3D extends Speed {
    protected int valueX;
    protected int valueY;
    protected int valueZ;

    public Speed3D(int x, int y, int z, SpeedUnit unit){
        super(unit);
        this.valueX = x;
        this.valueY = y;
        this.valueZ = z;
    }
    
    public Speed add(Speed delta){
        if(delta instanceof Speed3D){
            Speed3D delta3d = (Speed3D) delta;
            if(this.unit == delta3d.unit){
                return new Speed3D(this.valueX + delta3d.valueX, 
                    this.valueY + delta3d.valueY,
                    this.valueZ + delta3d.valueZ, 
                    this.unit);
            } else {
                return this;
            }
        } else {
            return this;
        }
    }
    
    public String toString(){
        return "(" + this.valueX + "," + this.valueY + "," + this.valueZ + ") " + this.unit;
    }
}

In [7]:
Speed speed1 = new Speed3D(10, 10, 10, SpeedUnit.KMH);
Speed speed2 = new Speed3D(30, 40, 50, SpeedUnit.KMH);

Speed speed3 = speed1.add(speed2);
System.out.println(speed3);

(40,50,60) KMH


In [8]:
class Speed3D extends Speed {
    protected int valueX;
    protected int valueY;
    protected int valueZ;

    public Speed3D(int x, int y, int z, SpeedUnit unit){
        super(unit);
        this.valueX = x;
        this.valueY = y;
        this.valueZ = z;
    }
    
    public Speed add(Speed delta){
        if(delta instanceof Speed1D){
            Speed1D delta1d = (Speed1D) delta;
            if(this.unit == delta1d.unit){
                return new Speed3D(this.valueX + delta1d.value, 
                    this.valueY + delta1d.value,
                    this.valueZ + delta1d.value, 
                    this.unit);
            } else {
                return this;
            }
        } else {
            return this;
        }
    }
    
    public String toString(){
        return "(" + this.valueX + "," + this.valueY + "," + this.valueZ + ") " + this.unit;
    }
}

In [9]:
Speed speed1 = new Speed3D(10, 10, 10, SpeedUnit.KMH);
Speed speed2 = new Speed1D(30, SpeedUnit.KMH);

Speed speed3 = speed1.add(speed2);
System.out.println(speed3);

(40,40,40) KMH


<img src="img/protected.png"/>

Nie ma dostępu pod warunkiem, że klasa Speed3D jest w innym pakiecie. Jeśli są w tym samym pakiecie, to klasa 
Speed3D ma nadal dostęp do pól klasy Speed1D, ponieważ dostęp chroniony implikuje dostęp pakietowy.

# Dostęp pakietowy

In [10]:
//package agh.cs.lecture;

class Speed {
    int value;
    SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}

In [11]:
//package agh.cs.lecture;

class SpaceProbe {
    private Speed speed;
    
    public void accelerate(Speed delta){
        if(this.speed.unit == delta.unit){           // dozwolone
            //..
        }
    }
}

In [ ]:
//package com.mycompany;

class SpaceShip {
    private Speed speed;

    public void accelerate(Speed delta){
        if(this.speed.unit == delta.unit){           // niedozwolone!
            //..
        }
    }
}

<img src="img/package_access.png"/>

# Podsumowanie modyfikatorów dostępu



| &nbsp;                                | private | default   | protected | public |
|---------------------------------------|---------|-----------|-----------|--------|
| ta sama klasa                         | +       | +         | +         | +      |
| klasa w tym samym pakiecie            | -       | +         | +         | +      |
| klasa dziedzicząca (z innego pakietu) | -       | -         | +         | +      |
| pozostałe klasy                       | -       | -         | -         | +      |

# Metody dostępowe - "gettery"

In [ ]:
class Speed {
    private int value;
    private SpeedUnit unit;
    
    public int getValue(){
        return this.value;
    }
    
    public SpeedUnit getUnit(){
        return this.unit;
    }
}

# Dostęp dziedzinowy

In [12]:
class Speed {
    private int value;
    private SpeedUnit unit;
    
    public int getValueInMS(){
        if(this.unit == SpeedUnit.MS){
            return value;
        } else {
            return convert(this.unit, SpeedUnit.MS, this.value);
        }
    }
    
    public int getValueInKMH(){
        //...
        return 0;
    }
    
    private int convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.KMH && to == SpeedUnit.MS) {
            return (int)Math.round(value / 3.6);
        } else {
            //...
            return 0;
        }
    }
}

# Metody dostępowe - "settery"

In [ ]:
class Speed {
    private int value;
    private SpeedUnit unit;
    
    public setValue(int value){
        this.value = value;
    }
    
    public setUnit(SpeedUnit unit){
        this.value = convert(this.unit, unit, this.value);
        this.unit = unit;
    }
}

To już nie jest "value object" !!

# Utrzymywanie jednolitej reprezentacji

In [ ]:
class Speed {
    private int valueInMs;
    private SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.valueInMs = convert(unit, SpeedUnit.MS, value);
        this.unit = unit;
    }
    
    public int getValue(){
        return convert(SpeedUnit.MS, this.unit, this.valueInMs);
    }
    
    public void setValue(int value){
        this.valueInMs = convert(unit, SpeedUnit.MS, value);
    }
    
    public void setUnit(SpeedUnit unit){
        this.unit = unit;
    }
    
    private int convert(SpeedUnit fromUnit, SpeedUnit toUnit, int value){
        //...
        return 0;
    }
}

# ValueObject - konwersja

In [ ]:
class Speed {
    private final int value;
    private final SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
    
    public Speed convertToMs(){
        if(this.unit == SpeedUnit.MS){
            return this;
        } else {
            return new Speed(convert(this.unit, SpeedUnit.MS, this.value), 
                             SpeedUnit.MS);
        }
    }
}

# Shadowing

In [13]:
enum SpeedUnit { MS, KMH }

In [14]:
class Speed {
    private SpeedUnit unit;
    
    public Speed(SpeedUnit unit){
        this.unit = unit;
    }
    
    public SpeedUnit getSuperUnit(){
        return this.unit;
    }
    
    public SpeedUnit getUnit(){
        return this.unit;
    }
}

In [16]:
class Speed1D extends Speed{
    private SpeedUnit unit = SpeedUnit.KMH;; // <-----!
    private int value;
    
    public Speed1D(int value, SpeedUnit unit){
        super(unit);
        this.value = value;
    }
    
    public SpeedUnit getUnit(){
        return this.unit;
    }
}

In [17]:
Speed1D speed1 = new Speed1D(10, SpeedUnit.MS);
System.out.println(speed1.getUnit());
System.out.println(speed1.getSuperUnit());

KMH
MS


Metoda jest wirtualna, ale to co "widzi" zależne jest od klasy, w której jest zdefiniowana.

<img src="img/shadowing.png"/>

In [18]:
class Speed {
    public SpeedUnit unit;
    
    public Speed(SpeedUnit unit){
        this.unit = unit;
    }
}

class Speed1D extends Speed {
    public SpeedUnit unit = SpeedUnit.KMH; // <-----!
    public int value;
    
    public Speed1D(SpeedUnit unit, int value){
        super(unit);
        this.value = value;
    }
}

Speed speed1 = new Speed1D(SpeedUnit.MS, 10);
System.out.println(speed1.unit);
Speed1D speed2 = (Speed1D) speed1;
System.out.println(speed2.unit);

MS
KMH


Atrybuty, w przeciwieństwie do metod, nie są wirtualne - to co "widzimy", zależne jest od zadeklarowanego typu zmiennej.

# Zmienne statyczne

In [ ]:
class Speed {
    private int value;
    private SpeedUnit unit;
    
    private double convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.MS && to == SpeedUnit.KMH){
            return value * 3.6;
        } else if(from == SpeedUnit.KMH && to == SpeedUnit.MS){
            return value / 3.6;
        } else {
            //...
            return 0;
        }
    }
}

In [ ]:
class Speed {
    private double ms2kmhRatio = 3.6;
    
    private int value;
    private SpeedUnit unit;
    
    private double convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.MS && to == SpeedUnit.KMH){
            return value * ms2kmhRatio;
        } else if(from == SpeedUnit.KMH && to == SpeedUnit.MS){
            return value / ms2kmhRatio;
        } else {
            //...
            return 0;
        }
    }
}

<img src="img/static1.png"/>

# Modyfikator `static`

In [ ]:
class Speed {
    private static final double ms2kmhRatio = 3.6;
    
    private int value;
    private SpeedUnit unit;
    
    private double convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.MS && to == SpeedUnit.KMH){
            return value * ms2kmhRatio;
        } else if(from == SpeedUnit.KMH && to == SpeedUnit.MS){
            return value / ms2kmhRatio;
        } else {
            //...
            return 0;
        }
    }
}

<img src="img/static2.png"/>

Modyfikator `static` oznacza, że dany atrubyt jest wspólny dla wszystkich obiektów, należących do klasy `Speed`.
Istnieje zatem jedna "kopia" tego atrybutu.

# Antywzorzec

In [ ]:
class SpaceShip {
    public static List<SpaceShip> ships = new LinkedList<>();
    
    public SpaceShip(){
        ships.add(this);
    }
}

Jednym z częstych błędów popełnianych w odniesieniu do modytifkatora `static` jest wykorzystanie go jako
"pojemnika" na wszystkie obiekty utworzone w programie. Jest to zdecydowanie niepoprawne użycie tego modyfikatora
z wielu względów. Utrudnia to m.in. testowanie kodu, ponieważ testy nie mogą być wykonywane równolegle.

In [20]:
class SpaceShip {}

class Space {
    private List<SpaceShip> ships = new LinkedList<>();
    
    public SpaceShip createShip(){
        SpaceShip ship = new SpaceShip(this);
        ships.add(ship);
        return ship;
    }
}

class SpaceShip {
    private Space space;
    
    public SpaceShip(Space space){
        this.space = space;
    }
}

In [21]:
Space thisWorld = new Space();
SpaceShip rocket = thisWorld.createShip();
SpaceShip spaceShuttle = thisWorld.createShip();

Space alienWorld = new Space();
SpaceShip ufo = alienWorld.createShip();

Jeśli chcemy mieć "pojemnik" na obiekty możemy np. utrudnić ich tworzenie poprzez uczynienie konstruktora 
niepublicznym. Wtedy jedynym sposobem na tworzenie nowych obiektów (o ile robią to klasy z innych pakietów)
jest pośrednictwo "fabryki", która sama zadba o to, żeby tworzone obiekty od razu trafiały do odpowiedniego
kontenera. Możemy jednak tworzyć wiele fabryk, co bardzo ułatwia np. testowanie kodu.